In [2]:
import os
import pandas as pd
from scipy.io import wavfile

In [3]:
path = "/Users/acabrera/dev/data/free-spoken-digit-dataset/recordings"

In [4]:
ents = os.listdir(path)

In [9]:
res = [] 
for e in ents:
    parts = e[0:-4].split("_")
    res.append({
        "label": int(parts[0]),
        "speaker": parts[1],
        "take": int(parts[2]),
        "id": e
    })
df = pd.DataFrame(res)

In [10]:
df.head()

,label,speaker,take,id
0,5,nicolas,9,5_nicolas_9.wav
1,3,yweweler,14,3_yweweler_14.wav
2,1,lucas,5,1_lucas_5.wav
3,2,george,44,2_george_44.wav
4,4,yweweler,38,4_yweweler_38.wav


In [12]:
df.to_csv('fsdd.csv', index=False)

In [5]:
import torch
import zipfile
import torchaudio
from glob import glob

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file('https://opus-codec.org/static/examples/samples/speech_orig.wav',
                               dst ='speech_orig.wav', progress=True)
test_files = glob("/Users/acabrera/dev/data/free-spoken-digit-dataset/recordings/0_yweweler_38.wav")
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]),
                            device=device)

output = model(input)
for example in output:
    print(decoder(example.cpu()))


Using cache found in /Users/acabrera/.cache/torch/hub/snakers4_silero-models_master
100.0%


zero


In [18]:
fs = [read_audio("/Users/acabrera/dev/data/free-spoken-digit-dataset/recordings/0_yweweler_38.wav"),read_audio("/Users/acabrera/dev/data/free-spoken-digit-dataset/recordings/0_yweweler_38.wav")]

In [20]:
input = prepare_model_input(fs,
                            device=device)

output = model(input)
for example in output:
    print(decoder(example.cpu()))

zero
zero


### Audio Processing

In [11]:
import librosa
import numpy as np

In [15]:
for ent in ents[0:5]:
    y, sr = librosa.load(os.path.join(path, ent))
    print(np.abs(y).mean())

0.0384884
0.0055082603
0.028301653
0.039489824
0.006017736
